In [7]:
import learn2learn as l2l
import torchvision
from learn2learn.data import TaskGenerator

mnist = torchvision.datasets.MNIST(root="/data", train=True, download=True)

mnist = l2l.data.MetaDataset(mnist)
task_generator = l2l.data.TaskGenerator(mnist,
                                        ways=3,
                                        classes=[0, 1, 4, 6, 8, 9],
                                        tasks=10)

ImportError: cannot import name 'TaskGenerator' from 'learn2learn.data' (/home/khanhi83/anaconda3/envs/PY3/lib/python3.7/site-packages/learn2learn/data/__init__.py)

In [5]:
batch = task_generator.train.sample()
print(batch.shape)

NameError: name 'task_generator' is not defined

In [1]:
import random
import numpy as np

import torch
from torch import nn, optimf

import learn2learn as l2l
from learn2learn.data.transforms import (NWays,
                                         KShots,
                                         LoadData,
                                         RemapLabels,
                                         ConsecutiveLabels)

In [51]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        adaptation_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(adaptation_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    evaluation_error = loss(predictions, evaluation_labels)
    evaluation_accuracy = accuracy(predictions, evaluation_labels)
    return evaluation_error, evaluation_accuracy

In [52]:
def main(
        ways=5,
        shots=5,
        meta_lr=0.003,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=100,
        cuda=True,
        seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda and torch.cuda.device_count():
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Create Tasksets using the benchmark interface
    tasksets = l2l.vision.benchmarks.get_tasksets('mini-imagenet',
                                                  train_samples=2*shots,
                                                  train_ways=ways,
                                                  test_samples=2*shots,
                                                  test_ways=ways,
                                                  root='~/data',
    )

    # Create model
    model = l2l.vision.models.MiniImagenetCNN(ways)
    model.to(device)
    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.Adam(maml.parameters(), meta_lr)
    loss = nn.CrossEntropyLoss(reduction='mean')

    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = tasksets.train.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = tasksets.validation.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
        print('\n')
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)
        print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
        print('Meta Valid Error', meta_valid_error / meta_batch_size)
        print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-testing loss
        learner = maml.clone()
        batch = tasksets.test.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           shots,
                                                           ways,
                                                           device)
        meta_test_error += evaluation_error.item()
        meta_test_accuracy += evaluation_accuracy.item()
    print('Meta Test Error', meta_test_error / meta_batch_size)
    print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)


if __name__ == '__main__':
    main()

Downloading: /home/hikmat/data/mini-imagenet-cache-train.pkl
Downloading: /home/hikmat/data/mini-imagenet-cache-validation.pkl
Downloading: /home/hikmat/data/mini-imagenet-cache-test.pkl


Iteration 0
Meta Train Error 6.996662646532059
Meta Train Accuracy 0.21749999094754457
Meta Valid Error 7.403078183531761
Meta Valid Accuracy 0.21374999172985554


Iteration 1
Meta Train Error 6.423653170466423
Meta Train Accuracy 0.2412499920465052
Meta Valid Error 6.596770539879799
Meta Valid Accuracy 0.2174999921116978


Iteration 2
Meta Train Error 5.23577856272459
Meta Train Accuracy 0.25374999246560037
Meta Valid Error 5.110181927680969
Meta Valid Accuracy 0.25499999430030584


Iteration 3
Meta Train Error 3.5246969014406204
Meta Train Accuracy 0.2599999934900552
Meta Valid Error 3.515685185790062
Meta Valid Accuracy 0.2687499914318323


Iteration 4
Meta Train Error 2.3865282982587814
Meta Train Accuracy 0.2737499945797026
Meta Valid Error 2.4741680808365345
Meta Valid Accuracy 0.26499999058432



Iteration 48
Meta Train Error 1.5190819390118122
Meta Train Accuracy 0.3374999896623194
Meta Valid Error 1.5073438249528408
Meta Valid Accuracy 0.3387499931268394


Iteration 49
Meta Train Error 1.51467365026474
Meta Train Accuracy 0.33874999172985554
Meta Valid Error 1.473341129720211
Meta Valid Accuracy 0.3474999889731407


Iteration 50
Meta Train Error 1.486496213823557
Meta Train Accuracy 0.3562499894760549
Meta Valid Error 1.4364779330790043
Meta Valid Accuracy 0.3674999908544123


Iteration 51
Meta Train Error 1.4418983608484268
Meta Train Accuracy 0.3749999883584678
Meta Valid Error 1.4291097149252892
Meta Valid Accuracy 0.3762499885633588


Iteration 52
Meta Train Error 1.4877872690558434
Meta Train Accuracy 0.352499988861382
Meta Valid Error 1.480625405907631
Meta Valid Accuracy 0.3812499907799065


Iteration 53
Meta Train Error 1.4677708260715008
Meta Train Accuracy 0.3662499887868762
Meta Valid Error 1.460544753819704
Meta Valid Accuracy 0.3562499904073775


Iteration 54
M



Iteration 98
Meta Train Error 1.40418154373765
Meta Train Accuracy 0.42374998750165105
Meta Valid Error 1.4440869614481926
Meta Valid Accuracy 0.38124998938292265


Iteration 99
Meta Train Error 1.321681885048747
Meta Train Accuracy 0.4462499883957207
Meta Valid Error 1.4518012218177319
Meta Valid Accuracy 0.3812499884516001
Meta Test Error 1.4991815760731697
Meta Test Accuracy 0.3562499890103936


In [ ]:
import learn2learn as l2l
for name in l2l.vision.benchmarks.list_tasksets():
    print(name)
    tasksets = l2l.vision.benchmarks.get_tasksets(name)

omniglot
Files already downloaded and verified
Files already downloaded and verified
mini-imagenet
tiered-imagenet


In [39]:
from torchmeta.datasets.helpers import cifar_fs, omniglot
from torchmeta.utils.data import BatchMetaDataLoader
SEED = 101

In [40]:
dataset = omniglot("data", ways=5, shots=5, test_shots=15, meta_train=True, download=True, seed=SEED)
dataloader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)

In [41]:
dataset = cifar_fs("data", ways=5, shots=5, test_shots=15, meta_train=True, download=True, seed=SEED)
dataloader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)

In [43]:



for batch in dataloader:
    train_inputs, train_targets = batch["train"]
    print('Train inputs shape: {0}'.format(train_inputs.shape))    # (16, 25, 1, 28, 28)
    print('Train targets shape: {0}'.format(train_targets.shape))  # (16, 25)

    test_inputs, test_targets = batch["test"]
    print('Test inputs shape: {0}'.format(test_inputs.shape))      # (16, 75, 1, 28, 28)
    print('Test targets shape: {0}'.format(test_targets.shape))    # (16, 75)
    break

Train inputs shape: torch.Size([16, 25, 3, 32, 32])
Train targets shape: torch.Size([16, 25])
Test inputs shape: torch.Size([16, 75, 3, 32, 32])
Test targets shape: torch.Size([16, 75])


In [44]:
import torch.nn as nn
from torchmeta.modules import (MetaModule, MetaSequential,
                               MetaConv2d, MetaLinear)
# from torchmeta.modules.utils import get_subdict
# from torchmeta.modules.utils import get_subdict

class Model(MetaModule):
    def __init__(self, in_channels, num_classes):
        super(Model, self).__init__()
        self.features = MetaSequential(MetaConv2d(in_channels, 64, 3),
                                       nn.ReLU(),
                                       nn.MaxPool2d(2))
        self.classifier = MetaLinear(64, num_classes)

    def forward(self, inputs, params=None):
#         features = self.features(inputs,
#                                  params=get_subdict(params, 'features'))
#         logits = self.classifier(features.view((inputs.size(0), -1)),
#                                  params=get_subdict(params, 'classifier'))
#         return logits
        return None

In [45]:
metaModel = Model(in_channels=32, num_classes=10)
print(metaModel)

Model(
  (features): MetaSequential(
    (0): MetaConv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): MetaLinear(in_features=64, out_features=10, bias=True)
)


In [5]:
import learn2learn as l2l
import torchvision
from torchvision import transforms
from learn2learn.data import TaskDataset


# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

dataset = torchvision.datasets.MNIST(root="data", train=True, download=True, transform=transform)
dataset = l2l.data.MetaDataset(dataset)

transforms = [
    l2l.data.transforms.NWays(dataset, n=2),
    l2l.data.transforms.KShots(dataset, k=5),
    l2l.data.transforms.LoadData(dataset),
]


taskset = TaskDataset(dataset, transforms, num_tasks=5)
for task in taskset:
    X, y = task
    print(y)
    


tensor([5, 5, 5, 5, 5, 9, 9, 9, 9, 9])
tensor([6, 6, 6, 6, 6, 3, 3, 3, 3, 3])
tensor([6, 6, 6, 6, 6, 3, 3, 3, 3, 3])
tensor([5, 5, 5, 5, 5, 8, 8, 8, 8, 8])
tensor([1, 1, 1, 1, 1, 9, 9, 9, 9, 9])


In [1]:

task_generator = l2l.data.TaskGenerator(mnist,
                                        ways=3,
                                        classes=[0, 1, 4, 6, 8, 9],
                                        tasks=10)
model = Net()
maml = l2l.algorithms.MAML(model, lr=1e-3, first_order=False)
opt = optim.Adam(maml.parameters(), lr=4e-3)

for iteration in range(num_iterations):
    learner = maml.clone()  # Creates a clone of model
    adaptation_task = task_generator.sample(shots=1)

    # Fast adapt
    for step in range(adaptation_steps):
        error = compute_loss(adaptation_task)
        learner.adapt(error)

    # Compute evaluation loss
    evaluation_task = task_generator.sample(shots=1,
                                            task=adaptation_task.sampled_task)
    evaluation_error = compute_loss(evaluation_task)

    # Meta-update the model parameters
    opt.zero_grad()
    evaluation_error.backward()
    opt.step()

AttributeError: module 'learn2learn.data' has no attribute 'TaskGenerator'

In [34]:
#!/usr/bin/env python3

import argparse
import random

import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

import learn2learn as l2l




In [49]:
Test = False
print(Test)
print(~Test)

True
-2


In [9]:
a = [1,2,3,4,5,6]
b = [6,5,4,3,2,1]
a = map(str, a)
print(a)
print(list(a))
for A in a:
    print(A)
    print(type(A))
# for A, B in enumerate(a,b):
#     print(A)

['1', '2', '3', '4', '5', '6']
